In [1]:
import numpy as np
import pandas as pd
import nltk
import Sastrawi
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.tag import CRFTagger
from collections import Counter
import warnings
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
warnings.filterwarnings("ignore")
import json
from nltk.tag import CRFTagger
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [3]:
factory_stopwords = StopWordRemoverFactory()
stopwords = factory_stopwords.get_stop_words()
print(stopwords)

['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua', 'ia', 'seperti', 'jika', 'jika', 'sehingga', 'kembali', 'dan', 'tidak', 'ini', 'karena', 'kepada', 'oleh', 'saat', 'harus', 'sementara', 'setelah', 'belum', 'kami', 'sekitar', 'bagi', 'serta', 'di', 'dari', 'telah', 'sebagai', 'masih', 'hal', 'ketika', 'adalah', 'itu', 'dalam', 'bisa', 'bahwa', 'atau', 'hanya', 'kita', 'dengan', 'akan', 'juga', 'ada', 'mereka', 'sudah', 'saya', 'terhadap', 'secara', 'agar', 'lain', 'anda', 'begitu', 'mengapa', 'kenapa', 'yaitu', 'yakni', 'daripada', 'itulah', 'lagi', 'maka', 'tentang', 'demi', 'dimana', 'kemana', 'pula', 'sambil', 'sebelum', 'sesudah', 'supaya', 'guna', 'kah', 'pun', 'sampai', 'sedangkan', 'selagi', 'sementara', 'tetapi', 'apakah', 'kecuali', 'sebab', 'selain', 'seolah', 'seraya', 'seterusnya', 'tanpa', 'agak', 'boleh', 'dapat', 'dsb', 'dst', 'dll', 'dahulu', 'dulunya', 'anu', 'demikian', 'tapi', 'ingin', 'juga', 'nggak', 'mari', 'nanti', 'melainkan', '

In [6]:
# single = pd.read_csv('single_annotator.csv')
# double = pd.read_csv('double_annotator_agree.csv')
# triple = pd.read_csv('triple_annotator_agree.csv')
# all_annos = pd.concat([single, double, triple],  ignore_index=True)

all_annos = pd.read_csv('all_data.csv')
all_annos

,kalimat_id,kata,sense,kalimat,preprocessed
0,336691,cerah,4801,Cuaca cerah adalah lazim sepanjang tahun.,cuaca cerah lazim panjang tahun
1,336270,cerah,4801,Gambar yang dihasilkan oleh layarnya cukup cer...,gambar hasil layar cukup cerah milik speaker m...
2,336555,cerah,4803,Masa depan yang cerah bagi pemuda berumur 20 d...,masa depan cerah pemuda umur prancis abad
3,336618,cerah,4801,"Cor Caroli (Alpha Canum Venaticorum), (nama le...",cor caroli alpha canum venaticorum nama lengka...
4,336613,cerah,4801,Sanders lebih menyukai cat air untuk Lilo deng...,sanders lebih suka cat air lilo maksud tampil ...
5,336406,cerah,4801,Ulleungdo memiliki iklim subtropis basah (Klas...,ulleungdo milik iklim subtropis basah klasifik...
6,336324,cerah,4801,Ikan hias kemungkinan besar sulit bertahan hid...,ikan hias mungkin besar sulit tahan hidup alam...
7,336401,cerah,4801,Sebuah entremet menandai berakhirnya sajian su...,buah entremet tanda akhir saji suatu set menu ...
8,336426,cerah,4801,Alangkah lega hatinya ketika ia mulai melihat ...,alangkah lega hati ia mulai lihat binatang ter...
9,337760,coklat,4703,Sisi atas tubuh berwarna coklat tembaga keemasan.,sisi atas tubuh warna coklat tembaga emas


In [7]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def tokenize_sentence(sentence):
    token = nltk.word_tokenize(sentence)
    tokenized_sentence = ' '.join(token)
    return tokenized_sentence

def normalisasi(sentence): #normalisasi 1 sentence
    normal_st= sentence.lower() #lowercase
    normal_st = re.sub('[^\x30-\x39^\x41-\x5A^\x61-\x7A^\s^-]','',normal_st) #buang punctuation dan karakter selain A-Z, a-z, dan 0-9
    normal_st = re.sub('\d','',normal_st)
    normal_st = re.sub('\s+', ' ', normal_st) # remove extra space
    normal_st = normal_st.strip() #trim depan belakang
    normal_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE) #regex huruf yang berulang kaya haiiii (untuk fitur unigram)
    normal_st = normal_regex.sub(r"\1\1", normal_st) #buang huruf yang berulang
    return normal_st

def remove_stopword(word,sentence,stopwords): #menghapus stopwords
    token = sentence.split()
    for idx_t, t in enumerate(token):
        if t != word and t in stopwords:
            del token[idx_t]
    token = ' '.join(token)
    return token

def stemming(word, sentence): #stemming sentence
    token = sentence.split()
    global factory, stemmer
    for idx_w, w in enumerate(token):
        if w != word:
            token[idx_w] = stemmer.stem(w)
            
    token = ' '.join(token)
    return token

def do_normalization(words, raw_sentence, stopwords): #normalisasi banyak sentence
    normalized_st = []
    count_sentences = 0
    for idx_st, st in enumerate(raw_sentence):
        tokenized_st= tokenize_sentence(st)
        normal_st = normalisasi(tokenized_st)
        normal_st = remove_stopword(words[idx_st], normal_st, stopwords)
        normal_st = stemming(words[idx_st],normal_st)
        normalized_st.append(normal_st)
        count_sentences += 1
        print("{} sentences done ".format(count_sentences))
        
    print("Normalization Done")
    return normalized_st

In [8]:
words = all_annos['kata'].tolist()
senses = all_annos['sense'].tolist()

# raw_st = all_annos['kalimat']
# prp_st = do_normalization(words,raw_st,stopwords)

prp_st = pd.read_csv('preprocessed.csv')['preprocessed'] #prp = preprocessed, st = sentences

In [9]:
#method ini berfungsi membuang sentences training yang mengandung lebih dari satu kata yang akan ditentukan sense-nya
def filter_sentences(prp_st, words, senses):
    corpus = []
    corresponding_words = []
    corresponding_senses = []
    data = prp_st

    for idx_st, st in enumerate(data):
        tmp = []
        token = st.split()
        cnt_find = 0
        for idx_w, w in enumerate(token):
            if w != words[idx_st]:
                tmp.append(w)
            else:
                cnt_find += 1
        if cnt_find == 1:
            corpus.append(' '.join(tmp))
            corresponding_words.append(words[idx_st])
            corresponding_senses.append(senses[idx_st])
    return corpus, corresponding_words, corresponding_senses

# corpus_train berisi sentences yang mana kata yang akan ditentukan sense-nya sudah 
# dibuang dan dipindahkan ke corresponding_words_train. correponding_senses_train berisi sense
# yang berpadanan dengan kata pada corresponding_words_train
corpus_train, corresponding_words_train, corresponding_senses_train = filter_sentences(prp_st, words, senses)

In [11]:
# group_sentences berisi dictionary yang key nya kata yang ingin dicari sense nya,
# dan value nya berupa list of sentences yang mengandung kata tsb
# group_senses berisi dictonary yang key nya kata yang ingin dicari sense nya,
# dan value nya berupa list of senses yang berpadanan dengan list of sentences pada group_sentences
def group_sentences_senses(corpus, corresponding_words, corresponding_senses):
    group_sentences = {}
    group_senses = {}
    for idx_w, w in enumerate(corresponding_words):
        if w not in group_sentences:
            group_sentences[w] = []
            group_senses[w] = []
        group_sentences[w].append(corpus[idx_w])
        group_senses[w].append(corresponding_senses[idx_w])
    return group_sentences, group_senses
group_sentences, group_senses = group_sentences_senses(corpus_train, corresponding_words_train, corresponding_senses_train)

print(group_sentences)
print(group_senses)

{'cerah': ['cuaca lazim panjang tahun', 'gambar hasil layar cukup milik speaker mampu hasil suara keras jernih', 'masa depan pemuda umur prancis abad', 'cor caroli alpha canum venaticorum nama lengkap cor caroli regis martyris bintang paling rasi canes venatici', 'sanders lebih suka cat air lilo maksud tampil warna buku cerita gambar', 'ulleungdo milik iklim subtropis basah klasifikasi iklim koppen cfa pada musim dingin hujan sangat deras cuaca sangat jarang', 'ikan hias mungkin besar sulit tahan hidup alam liar akibat warna-warninya sirip panjang', 'buah entremet tanda akhir saji suatu set menu dapat upa apa mulai frumenty sederhana macam bubur gandum warna bumbu rempah', 'alangkah lega hati ia mulai lihat binatang ternak liar atas ladang rumput burung bangan udara', 'magnitudo visual yupiter paling saat sedang oposisi yang paling rendah saat sedang konjungsi matahari', 'kota dapat sinar matahari panjang tahun milik rata hari tahun sedikit hujan', 'suhu rata musim panas sedikit lebih 

In [12]:
# data_test = pd.read_csv('testing_data.csv')
# words_test = data_test['word'].tolist()
# raw_st_test = data_test['kalimat']
# prp_st_test = do_normalization(words_test,raw_st_test,stopwords)
# data_test['preprocessed'] = prp_st_test
# data_test[['id','word','preprocessed']].to_csv('preprocessed_test.csv', index=False)

data_test = pd.read_csv('preprocessed_test.csv')
prp_st_test = data_test['preprocessed']
words_test = data_test['word']
id_st_test = data_test['id']

#method ini berfungsi menghapus kata yang ingin dicari sense nya pada kalimat sebelum kalimat tsb dimasukkan ke tf-idf
#vectorizer
def remove_words_test(prp_st_test, words_test):
    corpus = []
    data = prp_st_test

    for idx_st, st in enumerate(data):
        tmp = []
        token = st.split()
        for idx_w, w in enumerate(token):
            if w != words_test[idx_st]:
                tmp.append(w)
        corpus.append(' '.join(tmp))
    return corpus

corpus_test = remove_words_test(prp_st_test, words_test)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

In [14]:
y_pred_svm = []

#prediksi dengan metode tf-idf feature extraction, dilanjutkan dengan svm untuk menebak sense yang tepat
for idx_w, w in enumerate(words_test):
    
    vectorizer = TfidfVectorizer()
    vect = vectorizer.fit_transform(group_sentences[w]+[corpus_test[idx_w]])
    X_train = vect[:-1]
    y_train = group_senses[w]
    X_test = vect[-1]
    
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    
    y_pred_svm.append([id_st_test[idx_w],w,lin_clf.predict(X_test)[0]])
    
    print("{} sentences done".format(idx_w+1))

y_pred_svm = pd.DataFrame(y_pred)

1 sentences done
2 sentences done
3 sentences done
4 sentences done
5 sentences done
6 sentences done
7 sentences done
8 sentences done
9 sentences done
10 sentences done
11 sentences done
12 sentences done
13 sentences done
14 sentences done
15 sentences done
16 sentences done
17 sentences done
18 sentences done
19 sentences done
20 sentences done
21 sentences done
22 sentences done
23 sentences done
24 sentences done
25 sentences done
26 sentences done
27 sentences done
28 sentences done
29 sentences done
30 sentences done
31 sentences done
32 sentences done
33 sentences done
34 sentences done
35 sentences done
36 sentences done
37 sentences done
38 sentences done
39 sentences done
40 sentences done
41 sentences done
42 sentences done
43 sentences done
44 sentences done
45 sentences done
46 sentences done
47 sentences done
48 sentences done
49 sentences done
50 sentences done
51 sentences done
52 sentences done
53 sentences done
54 sentences done
55 sentences done
56 sentences done
5

444 sentences done
445 sentences done
446 sentences done
447 sentences done
448 sentences done
449 sentences done
450 sentences done
451 sentences done
452 sentences done
453 sentences done
454 sentences done
455 sentences done
456 sentences done
457 sentences done
458 sentences done
459 sentences done
460 sentences done
461 sentences done
462 sentences done
463 sentences done
464 sentences done
465 sentences done
466 sentences done
467 sentences done
468 sentences done
469 sentences done
470 sentences done
471 sentences done
472 sentences done
473 sentences done
474 sentences done
475 sentences done
476 sentences done
477 sentences done
478 sentences done
479 sentences done
480 sentences done
481 sentences done
482 sentences done
483 sentences done
484 sentences done
485 sentences done
486 sentences done
487 sentences done
488 sentences done
489 sentences done
490 sentences done
491 sentences done
492 sentences done
493 sentences done
494 sentences done
495 sentences done
496 sentence

880 sentences done
881 sentences done
882 sentences done
883 sentences done
884 sentences done
885 sentences done
886 sentences done
887 sentences done
888 sentences done
889 sentences done
890 sentences done
891 sentences done
892 sentences done
893 sentences done
894 sentences done
895 sentences done
896 sentences done
897 sentences done
898 sentences done
899 sentences done
900 sentences done
901 sentences done
902 sentences done
903 sentences done
904 sentences done
905 sentences done
906 sentences done
907 sentences done
908 sentences done
909 sentences done
910 sentences done
911 sentences done
912 sentences done
913 sentences done
914 sentences done
915 sentences done
916 sentences done
917 sentences done
918 sentences done
919 sentences done
920 sentences done
921 sentences done
922 sentences done
923 sentences done
924 sentences done
925 sentences done
926 sentences done
927 sentences done
928 sentences done
929 sentences done
930 sentences done
931 sentences done
932 sentence

1299 sentences done
1300 sentences done
1301 sentences done
1302 sentences done
1303 sentences done
1304 sentences done
1305 sentences done
1306 sentences done
1307 sentences done
1308 sentences done
1309 sentences done
1310 sentences done
1311 sentences done
1312 sentences done
1313 sentences done
1314 sentences done
1315 sentences done
1316 sentences done
1317 sentences done
1318 sentences done
1319 sentences done
1320 sentences done
1321 sentences done
1322 sentences done
1323 sentences done
1324 sentences done
1325 sentences done
1326 sentences done
1327 sentences done
1328 sentences done
1329 sentences done
1330 sentences done
1331 sentences done
1332 sentences done
1333 sentences done
1334 sentences done
1335 sentences done
1336 sentences done
1337 sentences done
1338 sentences done
1339 sentences done
1340 sentences done
1341 sentences done
1342 sentences done
1343 sentences done
1344 sentences done
1345 sentences done
1346 sentences done
1347 sentences done
1348 sentences done


1718 sentences done
1719 sentences done
1720 sentences done
1721 sentences done
1722 sentences done
1723 sentences done
1724 sentences done
1725 sentences done
1726 sentences done
1727 sentences done
1728 sentences done
1729 sentences done
1730 sentences done
1731 sentences done
1732 sentences done
1733 sentences done
1734 sentences done
1735 sentences done
1736 sentences done
1737 sentences done
1738 sentences done
1739 sentences done
1740 sentences done
1741 sentences done
1742 sentences done
1743 sentences done
1744 sentences done
1745 sentences done
1746 sentences done
1747 sentences done
1748 sentences done
1749 sentences done
1750 sentences done
1751 sentences done
1752 sentences done
1753 sentences done
1754 sentences done
1755 sentences done
1756 sentences done
1757 sentences done
1758 sentences done
1759 sentences done
1760 sentences done
1761 sentences done
1762 sentences done
1763 sentences done
1764 sentences done
1765 sentences done
1766 sentences done
1767 sentences done


...

7499 sentences done
7500 sentences done
7501 sentences done
7502 sentences done
7503 sentences done
7504 sentences done
7505 sentences done
7506 sentences done
7507 sentences done
7508 sentences done
7509 sentences done
7510 sentences done
7511 sentences done
7512 sentences done
7513 sentences done
7514 sentences done
7515 sentences done
7516 sentences done
7517 sentences done
7518 sentences done
7519 sentences done
7520 sentences done
7521 sentences done
7522 sentences done
7523 sentences done
7524 sentences done
7525 sentences done
7526 sentences done
7527 sentences done
7528 sentences done
7529 sentences done
7530 sentences done
7531 sentences done
7532 sentences done
7533 sentences done
7534 sentences done
7535 sentences done
7536 sentences done
7537 sentences done
7538 sentences done
7539 sentences done
7540 sentences done
7541 sentences done
7542 sentences done
7543 sentences done
7544 sentences done
7545 sentences done
7546 sentences done
7547 sentences done
7548 sentences done


7916 sentences done
7917 sentences done
7918 sentences done
7919 sentences done
7920 sentences done
7921 sentences done
7922 sentences done
7923 sentences done
7924 sentences done
7925 sentences done
7926 sentences done
7927 sentences done
7928 sentences done
7929 sentences done
7930 sentences done
7931 sentences done
7932 sentences done
7933 sentences done
7934 sentences done
7935 sentences done
7936 sentences done
7937 sentences done
7938 sentences done
7939 sentences done
7940 sentences done
7941 sentences done
7942 sentences done
7943 sentences done
7944 sentences done
7945 sentences done
7946 sentences done
7947 sentences done
7948 sentences done
7949 sentences done
7950 sentences done
7951 sentences done
7952 sentences done
7953 sentences done
7954 sentences done
7955 sentences done
7956 sentences done
7957 sentences done
7958 sentences done
7959 sentences done
7960 sentences done
7961 sentences done
7962 sentences done
7963 sentences done
7964 sentences done
7965 sentences done


8326 sentences done
8327 sentences done
8328 sentences done
8329 sentences done
8330 sentences done
8331 sentences done
8332 sentences done
8333 sentences done
8334 sentences done
8335 sentences done
8336 sentences done
8337 sentences done
8338 sentences done
8339 sentences done
8340 sentences done
8341 sentences done
8342 sentences done
8343 sentences done
8344 sentences done
8345 sentences done
8346 sentences done
8347 sentences done
8348 sentences done
8349 sentences done
8350 sentences done
8351 sentences done
8352 sentences done
8353 sentences done
8354 sentences done
8355 sentences done
8356 sentences done
8357 sentences done
8358 sentences done
8359 sentences done
8360 sentences done
8361 sentences done
8362 sentences done
8363 sentences done
8364 sentences done
8365 sentences done
8366 sentences done
8367 sentences done
8368 sentences done
8369 sentences done
8370 sentences done
8371 sentences done
8372 sentences done
8373 sentences done
8374 sentences done
8375 sentences done


8736 sentences done
8737 sentences done
8738 sentences done
8739 sentences done
8740 sentences done
8741 sentences done
8742 sentences done
8743 sentences done
8744 sentences done
8745 sentences done
8746 sentences done
8747 sentences done
8748 sentences done
8749 sentences done
8750 sentences done
8751 sentences done
8752 sentences done
8753 sentences done
8754 sentences done
8755 sentences done
8756 sentences done
8757 sentences done
8758 sentences done
8759 sentences done
8760 sentences done
8761 sentences done
8762 sentences done
8763 sentences done
8764 sentences done
8765 sentences done
8766 sentences done
8767 sentences done
8768 sentences done
8769 sentences done
8770 sentences done
8771 sentences done
8772 sentences done
8773 sentences done
8774 sentences done
8775 sentences done
8776 sentences done
8777 sentences done
8778 sentences done
8779 sentences done
8780 sentences done
8781 sentences done
8782 sentences done
8783 sentences done
8784 sentences done
8785 sentences done


NameError: name 'y_pred' is not defined

In [ ]:
# y_pred_svm.to_csv('prediction_tfidf.csv', index=False)

In [15]:
from sklearn.decomposition import TruncatedSVD

y_pred_svm_svd = []

# prediksi dengan metode tf-idf, dilanjutkan dengan svd feature extraction, 
# terakhir menggunakan svm menebak sense yang tepat
for idx_w, w in enumerate(words_test):
    
    vectorizer = TfidfVectorizer()
    vect = vectorizer.fit_transform(group_sentences[w]+[corpus_test[idx_w]])
    
    svd = TruncatedSVD(n_components=15, n_iter=20)
    
    vect = svd.fit_transform(vect)
    
    X_train = vect[:-1]
    
    y_train = group_senses[w]
    X_test = vect[-1]
    
    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train, y_train)
    
    y_pred_svm_svd.append([id_st_test[idx_w],w,lin_clf.predict([X_test])[0]])
    
    print("{} sentences done".format(idx_w+1))

y_pred_svm_svd = pd.DataFrame(y_pred)

1 sentences done
2 sentences done
3 sentences done
4 sentences done
5 sentences done
6 sentences done
7 sentences done
8 sentences done
9 sentences done
10 sentences done
11 sentences done
12 sentences done
13 sentences done
14 sentences done
15 sentences done
16 sentences done
17 sentences done
18 sentences done
19 sentences done
20 sentences done
21 sentences done
22 sentences done
23 sentences done
24 sentences done
25 sentences done
26 sentences done
27 sentences done
28 sentences done
29 sentences done
30 sentences done
31 sentences done
32 sentences done
33 sentences done
34 sentences done
35 sentences done
36 sentences done
37 sentences done
38 sentences done
39 sentences done
40 sentences done
41 sentences done
42 sentences done
43 sentences done
44 sentences done
45 sentences done
46 sentences done
47 sentences done
48 sentences done
49 sentences done
50 sentences done
51 sentences done
52 sentences done
53 sentences done
54 sentences done
55 sentences done
56 sentences done
5

438 sentences done
439 sentences done
440 sentences done
441 sentences done
442 sentences done
443 sentences done
444 sentences done
445 sentences done
446 sentences done
447 sentences done
448 sentences done
449 sentences done
450 sentences done
451 sentences done
452 sentences done
453 sentences done
454 sentences done
455 sentences done
456 sentences done
457 sentences done
458 sentences done
459 sentences done
460 sentences done
461 sentences done
462 sentences done
463 sentences done
464 sentences done
465 sentences done
466 sentences done
467 sentences done
468 sentences done
469 sentences done
470 sentences done
471 sentences done
472 sentences done
473 sentences done
474 sentences done
475 sentences done
476 sentences done
477 sentences done
478 sentences done
479 sentences done
480 sentences done
481 sentences done
482 sentences done
483 sentences done
484 sentences done
485 sentences done
486 sentences done
487 sentences done
488 sentences done
489 sentences done
490 sentence

870 sentences done
871 sentences done
872 sentences done
873 sentences done
874 sentences done
875 sentences done
876 sentences done
877 sentences done
878 sentences done
879 sentences done
880 sentences done
881 sentences done
882 sentences done
883 sentences done
884 sentences done
885 sentences done
886 sentences done
887 sentences done
888 sentences done
889 sentences done
890 sentences done
891 sentences done
892 sentences done
893 sentences done
894 sentences done
895 sentences done
896 sentences done
897 sentences done
898 sentences done
899 sentences done
900 sentences done
901 sentences done
902 sentences done
903 sentences done
904 sentences done
905 sentences done
906 sentences done
907 sentences done
908 sentences done
909 sentences done
910 sentences done
911 sentences done
912 sentences done
913 sentences done
914 sentences done
915 sentences done
916 sentences done
917 sentences done
918 sentences done
919 sentences done
920 sentences done
921 sentences done
922 sentence

1289 sentences done
1290 sentences done
1291 sentences done
1292 sentences done
1293 sentences done
1294 sentences done
1295 sentences done
1296 sentences done
1297 sentences done
1298 sentences done
1299 sentences done
1300 sentences done
1301 sentences done
1302 sentences done
1303 sentences done
1304 sentences done
1305 sentences done
1306 sentences done
1307 sentences done
1308 sentences done
1309 sentences done
1310 sentences done
1311 sentences done
1312 sentences done
1313 sentences done
1314 sentences done
1315 sentences done
1316 sentences done
1317 sentences done
1318 sentences done
1319 sentences done
1320 sentences done
1321 sentences done
1322 sentences done
1323 sentences done
1324 sentences done
1325 sentences done
1326 sentences done
1327 sentences done
1328 sentences done
1329 sentences done
1330 sentences done
1331 sentences done
1332 sentences done
1333 sentences done
1334 sentences done
1335 sentences done
1336 sentences done
1337 sentences done
1338 sentences done


1701 sentences done
1702 sentences done
1703 sentences done
1704 sentences done
1705 sentences done
1706 sentences done
1707 sentences done
1708 sentences done
1709 sentences done
1710 sentences done
1711 sentences done
1712 sentences done
1713 sentences done
1714 sentences done
1715 sentences done
1716 sentences done
1717 sentences done
1718 sentences done
1719 sentences done
1720 sentences done
1721 sentences done
1722 sentences done
1723 sentences done
1724 sentences done
1725 sentences done
1726 sentences done
1727 sentences done
1728 sentences done
1729 sentences done
1730 sentences done
1731 sentences done
1732 sentences done
1733 sentences done
1734 sentences done
1735 sentences done
1736 sentences done
1737 sentences done
1738 sentences done
1739 sentences done
1740 sentences done
1741 sentences done
1742 sentences done
1743 sentences done
1744 sentences done
1745 sentences done
1746 sentences done
1747 sentences done
1748 sentences done
1749 sentences done
1750 sentences done


...

7460 sentences done
7461 sentences done
7462 sentences done
7463 sentences done
7464 sentences done
7465 sentences done
7466 sentences done
7467 sentences done
7468 sentences done
7469 sentences done
7470 sentences done
7471 sentences done
7472 sentences done
7473 sentences done
7474 sentences done
7475 sentences done
7476 sentences done
7477 sentences done
7478 sentences done
7479 sentences done
7480 sentences done
7481 sentences done
7482 sentences done
7483 sentences done
7484 sentences done
7485 sentences done
7486 sentences done
7487 sentences done
7488 sentences done
7489 sentences done
7490 sentences done
7491 sentences done
7492 sentences done
7493 sentences done
7494 sentences done
7495 sentences done
7496 sentences done
7497 sentences done
7498 sentences done
7499 sentences done
7500 sentences done
7501 sentences done
7502 sentences done
7503 sentences done
7504 sentences done
7505 sentences done
7506 sentences done
7507 sentences done
7508 sentences done
7509 sentences done


7872 sentences done
7873 sentences done
7874 sentences done
7875 sentences done
7876 sentences done
7877 sentences done
7878 sentences done
7879 sentences done
7880 sentences done
7881 sentences done
7882 sentences done
7883 sentences done
7884 sentences done
7885 sentences done
7886 sentences done
7887 sentences done
7888 sentences done
7889 sentences done
7890 sentences done
7891 sentences done
7892 sentences done
7893 sentences done
7894 sentences done
7895 sentences done
7896 sentences done
7897 sentences done
7898 sentences done
7899 sentences done
7900 sentences done
7901 sentences done
7902 sentences done
7903 sentences done
7904 sentences done
7905 sentences done
7906 sentences done
7907 sentences done
7908 sentences done
7909 sentences done
7910 sentences done
7911 sentences done
7912 sentences done
7913 sentences done
7914 sentences done
7915 sentences done
7916 sentences done
7917 sentences done
7918 sentences done
7919 sentences done
7920 sentences done
7921 sentences done


8283 sentences done
8284 sentences done
8285 sentences done
8286 sentences done
8287 sentences done
8288 sentences done
8289 sentences done
8290 sentences done
8291 sentences done
8292 sentences done
8293 sentences done
8294 sentences done
8295 sentences done
8296 sentences done
8297 sentences done
8298 sentences done
8299 sentences done
8300 sentences done
8301 sentences done
8302 sentences done
8303 sentences done
8304 sentences done
8305 sentences done
8306 sentences done
8307 sentences done
8308 sentences done
8309 sentences done
8310 sentences done
8311 sentences done
8312 sentences done
8313 sentences done
8314 sentences done
8315 sentences done
8316 sentences done
8317 sentences done
8318 sentences done
8319 sentences done
8320 sentences done
8321 sentences done
8322 sentences done
8323 sentences done
8324 sentences done
8325 sentences done
8326 sentences done
8327 sentences done
8328 sentences done
8329 sentences done
8330 sentences done
8331 sentences done
8332 sentences done


8693 sentences done
8694 sentences done
8695 sentences done
8696 sentences done
8697 sentences done
8698 sentences done
8699 sentences done
8700 sentences done
8701 sentences done
8702 sentences done
8703 sentences done
8704 sentences done
8705 sentences done
8706 sentences done
8707 sentences done
8708 sentences done
8709 sentences done
8710 sentences done
8711 sentences done
8712 sentences done
8713 sentences done
8714 sentences done
8715 sentences done
8716 sentences done
8717 sentences done
8718 sentences done
8719 sentences done
8720 sentences done
8721 sentences done
8722 sentences done
8723 sentences done
8724 sentences done
8725 sentences done
8726 sentences done
8727 sentences done
8728 sentences done
8729 sentences done
8730 sentences done
8731 sentences done
8732 sentences done
8733 sentences done
8734 sentences done
8735 sentences done
8736 sentences done
8737 sentences done
8738 sentences done
8739 sentences done
8740 sentences done
8741 sentences done
8742 sentences done


NameError: name 'y_pred' is not defined

In [ ]:
# y_pred_svm_svd.to_csv('prediction_tfidf_svd.csv', index=False)

In [16]:
y_pred_svm

[[13, 'asing', '5302'],
 [19, 'asing', '5302'],
 [41, 'asing', '5302'],
 [44, 'asing', '5302'],
 [121, 'asing', '5302'],
 [138, 'asing', '5302'],
 [139, 'asing', '5302'],
 [140, 'asing', '5302'],
 [188, 'asing', '5302'],
 [190, 'asing', '5302'],
 [205, 'asing', '5302'],
 [242, 'asing', '5302'],
 [286, 'asing', '5302'],
 [311, 'asing', '5302'],
 [313, 'asing', '5302'],
 [353, 'asing', '5302'],
 [412, 'asing', '5302'],
 [419, 'asing', '5302'],
 [453, 'asing', '5302'],
 [465, 'asing', '5302'],
 [524, 'asing', '5302'],
 [535, 'asing', '5302'],
 [553, 'asing', '5302'],
 [587, 'asing', '5302'],
 [663, 'asing', '5302'],
 [704, 'asing', '5302'],
 [738, 'asing', '5302'],
 [775, 'asing', '5302'],
 [843, 'asing', '5302'],
 [851, 'asing', '5302'],
 [964, 'asing', '5302'],
 [1015, 'asing', '5302'],
 [1022, 'asing', '5302'],
 [1076, 'asing', '5302'],
 [1102, 'asing', '5302'],
 [1132, 'asing', '5302'],
 [1167, 'asing', '5302'],
 [1241, 'asing', '5302'],
 [1245, 'asing', '5302'],
 [1309, 'asing', '530

In [17]:
y_pred_svm_svd

[[13, 'asing', '5302'],
 [19, 'asing', '5302'],
 [41, 'asing', '5302'],
 [44, 'asing', '5302'],
 [121, 'asing', '5302'],
 [138, 'asing', '5302'],
 [139, 'asing', '5302'],
 [140, 'asing', '5302'],
 [188, 'asing', '5302'],
 [190, 'asing', '5302'],
 [205, 'asing', '5302'],
 [242, 'asing', '5302'],
 [286, 'asing', '5302'],
 [311, 'asing', '5302'],
 [313, 'asing', '5302'],
 [353, 'asing', '5302'],
 [412, 'asing', '5302'],
 [419, 'asing', '5302'],
 [453, 'asing', '5302'],
 [465, 'asing', '5302'],
 [524, 'asing', '5302'],
 [535, 'asing', '5302'],
 [553, 'asing', '5302'],
 [587, 'asing', '5302'],
 [663, 'asing', '5302'],
 [704, 'asing', '5302'],
 [738, 'asing', '5302'],
 [775, 'asing', '5302'],
 [843, 'asing', '5302'],
 [851, 'asing', '5302'],
 [964, 'asing', '5302'],
 [1015, 'asing', '5302'],
 [1022, 'asing', '5302'],
 [1076, 'asing', '5302'],
 [1102, 'asing', '5302'],
 [1132, 'asing', '5302'],
 [1167, 'asing', '5302'],
 [1241, 'asing', '5302'],
 [1245, 'asing', '5302'],
 [1309, 'asing', '530